In [ ]:
"""
(C) Yu-Hao Chen (2023)

This is the accompanying code for the paper
'Making the zeroth-order process fidelity independent of state preparation and measurement errors'
Yu-Hao Chen, Renata Wong, Hi-Sheng Goan
"""



import numpy as np
import matplotlib.pyplot as plt
import itertools
from qutip import sigmax, sigmay, sigmaz, qeye, Qobj, tensor, ket2dm
from qutip.measurement import measure_observable

from qutip import (Qobj, basis, gate_expand_1toN, qeye,
                   sigmax, sigmay, sigmaz, snot, tensor, rx, ry, rz, cnot, cphase)

from scipy import linalg

import copy
import itertools
from typing import List, Union

from qiskit import IBMQ, Aer, execute, QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.utils import QuantumInstance 

from qiskit.circuit import Parameter
from qiskit.compiler import transpile


# Import general libraries (needed for functions)
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import display

# Import Qiskit classes 
import qiskit
from qiskit import assemble, transpile
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb
from qiskit_experiments.library import StandardRB, InterleavedRB
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment
import qiskit.circuit.library as circuits
import qiskit.quantum_info as qi

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.providers.fake_provider import FakeParis
from qiskit.opflow import I, X, Y, Z
from qiskit_aer import QasmSimulator

backend = AerSimulator.from_backend(FakeParis())



# Packages related to state preparation and measurement (SPAM) errors
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import Kraus, SuperOp, DensityMatrix, state_fidelity, process_fidelity, average_gate_fidelity
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)


from typing import List, Optional, Union
from warnings import warn
from numpy.random import RandomState


from qiskit.quantum_info import Clifford
from qiskit.quantum_info import random_clifford
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import process_fidelity
from qiskit.extensions import RXGate, XGate, CXGate

from qiskit.circuit import QuantumCircuit, Instruction
from qiskit.quantum_info import Clifford, CNOTDihedral
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.transpiler.passes import RemoveBarriers





def get_parity(n):
    parity = 0
    while n:
        parity = ~parity
        n = n & (n - 1)
    return parity


def expectation_value_from_counts(counts, nqubits, nshots):
    exp_val = 0
    for x in map(''.join, itertools.product('01', repeat=nqubits)):
        if x in counts:   # making sure that x is in the output as the counts dictionary contains no values with 0 occurrence
            if get_parity(int(x,2)) == -1:
                exp_val = exp_val - counts[x]
            if get_parity(int(x,2)) == 0:
                exp_val = exp_val + counts[x]
    return exp_val/nshots


#from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error


depolarizing_model = NoiseModel()

# add depolarizing noise to 3 qubits u gates
error_d = depolarizing_error(0.12, 1) # 0.018
# 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11, 0.12]
error_d_cx = depolarizing_error(0.12, 2)
#depolarizing_model.add_all_qubit_quantum_error(error_d, "reset")
#depolarizing_model.add_all_qubit_quantum_error(error_d, "u")
#depolarizing_model.add_all_qubit_quantum_error(error_d, "measure")
depolarizing_model.add_all_qubit_quantum_error(error_d_cx, "cz")
noise_simulator = AerSimulator(noise_model=depolarizing_model)
noise_simulator

Initial_noise = SuperOp(error_d).to_instruction()
Initial_noise.label = "noise"

In [ ]:
# Generating circuits with/without SPAM errors for channel noise scaling method where the initial states are the SIC-states

INPUT_STATE = [[0, 0, 0], [2 * np.arccos(np.sqrt(1/3)), 0, 0], [2 * np.arccos(np.sqrt(1/3)), 2 * np.pi/3, 0],
                     [2 * np.arccos(np.sqrt(1/3)), 4 * np.pi/3, 0]]
nqubits = 3
Circuit_list = [1]


def generate_circuits_SAPM(nqubits, Circuit_list):
    
    params = ParameterVector('p', 2 * 3 * nqubits)
    half_len_params = int(len(params)/2)

    qr = QuantumRegister(nqubits, 'q')
    cr = ClassicalRegister(nqubits, 'c')
    
    circuits = []
    state_prep = []
    
    measur_list = []
    rb_circ_lists = []
    inputs = [''.join(i) for i in itertools.product('0123', repeat=nqubits)]
    paulis = [''.join(i) for i in itertools.product('IXYZ', repeat=nqubits)]
    
    
    # generating circuits for each combination of input state and Pauli operator
    
    for digit_string in inputs:

        qc = QuantumCircuit(qr, cr)

        # append input states
        for digit, qubit in zip(digit_string, range(qc.num_qubits)):
            qc.u(INPUT_STATE[int(digit)][0], INPUT_STATE[int(digit)][1], INPUT_STATE[int(digit)][2], qubit)
        qc.barrier()
        
        # for state preparation error
        for n in range(nqubits):
            degrees = np.random.normal(0, np.sqrt(5), size=(1, 3))
            qc.u(degrees[0][0]*(np.pi)/180, degrees[0][1]*(np.pi)/180, degrees[0][2]*(np.pi)/180, n)

        qc.barrier()
        state_prep.append(qc)
        
        
    for pauli in paulis:
        qc_m = QuantumCircuit(qr, cr)
        #qc_m.barrier()
        
        for single_pauli, qubit in zip(pauli, range(qc_m.num_qubits)):
            if single_pauli == 'I':
                qc_m.reset(qubit)
            if single_pauli == 'X':
                qc_m.h(qubit)
            if single_pauli == 'Y':
                qc_m.h(qubit)
                qc_m.p(-np.pi/2, qubit)
            if single_pauli == 'Z':
                pass
        
        qc_m.measure(qr, cr)
        measur_list.append(qc_m)
    return state_prep, measur_list #circuits, rb_circ_lists
        


def generate_circuits(nqubits, Circuit_list):
    
    params = ParameterVector('p', 2 * 3 * nqubits)
    half_len_params = int(len(params)/2)

    qr = QuantumRegister(nqubits, 'q')
    cr = ClassicalRegister(nqubits, 'c')
    
    circuits = []
    state_prep = []
    measur_list = []
    rb_circ_lists = []
    inputs = [''.join(i) for i in itertools.product('0123', repeat=nqubits)]
    paulis = [''.join(i) for i in itertools.product('IXYZ', repeat=nqubits)]
    
    
    # generating circuits for each combination of input state and Pauli operator
    
    for digit_string in inputs:

        qc = QuantumCircuit(qr, cr)

        # append input states
        for digit, qubit in zip(digit_string, range(qc.num_qubits)):
            qc.u(INPUT_STATE[int(digit)][0], INPUT_STATE[int(digit)][1], INPUT_STATE[int(digit)][2], qubit)
        qc.barrier()
        
        # for state preparation error
        #for n in range(nqubits):
        #    degrees = np.random.normal(0, np.sqrt(5), size=(1, 3))
        #    qc.u(degrees[0][0]*(np.pi)/180, degrees[0][1]*(np.pi)/180, degrees[0][2]*(np.pi)/180, n)

        #qc.barrier()
        state_prep.append(qc)
        
    for pauli in paulis:
        qc_m = QuantumCircuit(qr, cr)
        #qc_m.barrier()
        
        for single_pauli, qubit in zip(pauli, range(qc_m.num_qubits)):
            if single_pauli == 'I':
                qc_m.reset(qubit)
            if single_pauli == 'X':
                qc_m.h(qubit)
            if single_pauli == 'Y':
                qc_m.h(qubit)
                qc_m.p(-np.pi/2, qubit)
            if single_pauli == 'Z':
                pass
        
        qc_m.measure(qr, cr)
        measur_list.append(qc_m)
    return state_prep, measur_list#circuits, rb_circ_lists
     
    


circuits_rb_SPAM = generate_circuits_SAPM(nqubits, Circuit_list)[0]
measure_SPAM = generate_circuits_SAPM(nqubits, Circuit_list)[1]

circuits_rb = generate_circuits(nqubits, Circuit_list)[0]
measure = generate_circuits(nqubits, Circuit_list)[1]




In [ ]:
# Compose the whole circuit

nqubits=3
circuit_layers = [1, 10, 20, 50, 75, 100, 135, 180]
main_gate_lists = []
for r in circuit_layers:
    qc_main_gate = QuantumCircuit(nqubits)
    for _ in range(r):
        for k in range(2):
            for i in range(nqubits-1):
                qc_main_gate.cz(i, i+1)
                #qc_main_gate.cx(i,i+1)
        qc_main_gate.barrier()
    main_gate_lists.append(qc_main_gate)
    
    
def direRB_zero_fidelity_SPAM(circuits_rb_SPAM, measure_SPAM):
    DRB_state_lists = []
    for i in main_gate_lists:
        DRB_state_list = []
        for j in circuits_rb_SPAM:
            cir = j.compose(i)
            DRB_state_list.append(cir)
        DRB_state_lists.append(DRB_state_list)
        
    DRB_circuit_lists = []
    for i in DRB_state_lists:
        DRB_circuit_list = []
        for k in i:
            for j in measure_SPAM:
                cir = k.compose(j)
                DRB_circuit_list.append(cir)
        DRB_circuit_lists.append(DRB_circuit_list)

    
    return DRB_circuit_lists

def direRB_zero_fidelity(circuits_rb, measure):
    DRB_state_lists = []
    for i in main_gate_lists:
        DRB_state_list = []
        for j in circuits_rb:
            cir = j.compose(i)
            DRB_state_list.append(cir)
        DRB_state_lists.append(DRB_state_list)
        
    DRB_circuit_lists = []
    for i in DRB_state_lists:
        DRB_circuit_list = []
        for k in i:
            for j in measure:
                cir = k.compose(j)
                DRB_circuit_list.append(cir)
        DRB_circuit_lists.append(DRB_circuit_list)

    
    return DRB_circuit_lists


Cir_DRB_SPAM = direRB_zero_fidelity_SPAM(circuits_rb_SPAM, measure_SPAM)
Cir_DRB = direRB_zero_fidelity(circuits_rb, measure)

In [ ]:
# Define the noise medel for simulation

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile


noise_model = NoiseModel()

readout_1 = 0.003
readout_0 = 0.005
readout_error = ReadoutError([[1-readout_1, readout_1], [readout_0, 1-readout_0]])

dep_2 = depolarizing_error(0.01, 2) # 0.01
dep = depolarizing_error(0.003, 1) # 0.003

#noise_model.add_all_qubit_quantum_error(dep_2, "cx")
#noise_model.add_all_qubit_quantum_error(dep_2, "cz")
noise_model.add_all_qubit_quantum_error(error_gate2, "cz")
noise_model.add_all_qubit_quantum_error(error_gate2, "cx")

noise_model.add_readout_error(readout_error, [0])
noise_model.add_readout_error(readout_error, [1])
noise_model.add_readout_error(readout_error, [2])
noise_model.add_readout_error(readout_error, [3])

In [ ]:
# Calculate the expectation values of our circuits

import time

start_time = time.time()


from qiskit_aer import QasmSimulator
def expectation_values(nqubits, noisy=False):
    params = np.linspace(0, 0, 6 * nqubits)
    if noisy == True:
        #backend = noise_simulator  #FakeManilaV2()
        noise_simulator = AerSimulator(noise_model=noise_model)
        nshots = 1024

        Counts = []
        expvals = []
        #circuits = generate_circuits_SAPM(nqubits, Circuit_list)[1][0]#generate_circuits(nqubits)
        #circuits = Cir_DRB_SPAM
        circuit_values_lists = []
        for circuits in Cir_DRB_SPAM:
            circuit_values_SPAM_list = []
            #circuits = Cir_DRB_SPAM[i]
            for circuit in circuits:

                #circuit.assign_parameters(params, inplace=True)
                backend = transpile(circuit, noise_simulator)
                #results = execute(circuit, backend, shots=nshots).result()
                results = noise_simulator.run(backend).result()
                counts = results.get_counts(circuit)
                counts_dict = counts
                expval = expectation_value_from_counts(counts, nqubits, nshots)
                Counts.append(counts)
                expvals.append(expval)
                circuit_values_SPAM_list.append(expval)
            circuit_values_lists.append(circuit_values_SPAM_list)
        
    if noisy == False:
        backend = QasmSimulator(method='density_matrix') #Aer.get_backend('qasm_simulator')
    
        nshots = 1024

        Counts = []
        expvals = []
        #circuits = generate_circuits(nqubits, Circuit_list)[1][0]#generate_circuits(nqubits)
        #circuits = Cir_DRB
        circuit_values_lists = []
        for circuits in Cir_DRB:
            circuit_values_list = []
            #circuits = Cir_DRB[i]
            for circuit in circuits:

                #circuit.assign_parameters(params, inplace=True)
                results = execute(circuit, backend, shots=nshots).result()
                counts = results.get_counts(circuit)
                expval = expectation_value_from_counts(counts, nqubits, nshots)
                Counts.append(counts)
                expvals.append(expval)
                circuit_values_list.append(expval)
            circuit_values_lists.append(circuit_values_list)

    return circuit_values_lists, expvals
exp_SPAM = expectation_values(nqubits, noisy=True)[0]
exp = expectation_values(nqubits, noisy=False)[0]
elapsed_time = time.time() - start_time
print("Consuming time = ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
# Calculate the results of zero-fidelity 

DRB_zero_fidelity = 0.0

Value_lists = []
d = 2**nqubits
for i in range(len(circuit_layers)):
    Value_list = []
    for j in range(len(exp_SPAM[i])):
        DRB_zero_fidelity = (exp_SPAM[i][j] * exp[i][j])
        Value_list.append(DRB_zero_fidelity)
    sum_ = sum(Value_list)
        #Value_lists.append(sum_)
    Value_lists.append(sum_)
Value_lists